In [2]:
from treelib import Node, Tree

tree = Tree()

tree.create_node("Harry", "harry")  # No parent means its the root node
tree.create_node("Jane",  "jane"   , parent="harry")
tree.create_node("Bill",  "bill"   , parent="harry")
tree.create_node("Diane", "diane"  , parent="jane")
tree.create_node("Mary",  "mary"   , parent="diane")
tree.create_node("Mark",  "mark"   , parent="jane")

tree.show()

ModuleNotFoundError: No module named 'treelib'

In [3]:
c=2


In [4]:
c

2

In [5]:
! pip install treelib

  Created wheel for treelib: filename=treelib-1.6.1-py3-none-any.whl size=18386 sha256=a0dd4d5bba993fb75428cdbcd5f8e55e4d5ec64651030472e32aeebd02bf27d4
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\43\de\32\fd270cc5f06de2bebcdaa666e9350b95f54d983212a7aba9ad
Successfully built treelib


In [6]:
from treelib import Node, Tree

In [7]:
tree = Tree()

tree.create_node("Harry", "harry")  # No parent means its the root node
tree.create_node("Jane",  "jane"   , parent="harry")
tree.create_node("Bill",  "bill"   , parent="harry")
tree.create_node("Diane", "diane"  , parent="jane")
tree.create_node("Mary",  "mary"   , parent="diane")
tree.create_node("Mark",  "mark"   , parent="jane")

tree.show()

Harry
├── Bill
└── Jane
    ├── Diane
    │   └── Mary
    └── Mark



In [8]:
tree.to_graphviz()

digraph tree {
	"harry" [label="Harry", shape=circle]
	"bill" [label="Bill", shape=circle]
	"jane" [label="Jane", shape=circle]
	"diane" [label="Diane", shape=circle]
	"mark" [label="Mark", shape=circle]
	"mary" [label="Mary", shape=circle]

	"harry" -> "jane"
	"harry" -> "bill"
	"jane" -> "diane"
	"jane" -> "mark"
	"diane" -> "mary"
}


In [12]:
import pandas as pd
import math
import numpy as np

data = pd.read_csv('data.csv')
features = [feat for feat in data]
features.remove('Answer')
print(data)
print(features)


class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

def entropy(examples):
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        if row["Answer"] == "yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        return 0.0
    else:
        p = pos / (pos + neg)
        n = neg / (pos + neg)
        return -(p * math.log(p, 2) + n * math.log(n, 2))

def info_gain(examples, attr):
    gain = entropy(examples)
    print ("\n",attr,"gain =",gain)
    uniq = np.unique(examples[attr])
    
    
    for u in uniq:
        subdata = examples[examples[attr] == u]
        #print ("\n subdata = \n",subdata)
        sub_e = entropy(subdata)
        gain -= (float(len(subdata)) / float(len(examples))) * sub_e
        print ("\n ",u,"gain =",gain)
    return gain


def ID3(examples, attrs):
    root = Node()

    max_gain = 0
    max_feat = ""
    for feature in attrs:
        #print ("\n",examples)
        gain = info_gain(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value = max_feat
    print ("\nMax feature attr",max_feat)
    uniq = np.unique(examples[max_feat])
    print ("\n",uniq)
    for u in uniq:
        print ("\n",u)
        subdata = examples[examples[max_feat] == u]
        #print ("\n",subdata)
        if entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["Answer"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)
    return root


def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)



root = ID3(data, features)
printTree(root)

     Outlook Temperature Humidity    Wind Answer
0      sunny         hot     high    weak     no
1      sunny         hot     high  strong     no
2   overcast         hot     high    weak    yes
3       rain        mild     high    weak    yes
4       rain        cool   normal    weak    yes
5       rain        cool   normal  strong     no
6   overcast        cool   normal  strong    yes
7      sunny        mild     high    weak     no
8      sunny        cool   normal    weak    yes
9       rain        mild   normal    weak    yes
10     sunny        mild   normal  strong    yes
11  overcast        mild     high  strong    yes
12  overcast         hot   normal    weak    yes
13      rain        mild     high  strong     no
['Outlook', 'Temperature', 'Humidity', 'Wind']

 Outlook gain = 0.9402859586706309

  overcast gain = 0.9402859586706309

  rain gain = 0.593517889222535

  sunny gain = 0.2467498197744391

 Temperature gain = 0.9402859586706309

  cool gain = 0.7084922088251644

 

In [14]:
root.children

In [20]:
def generateTree(node,actualTree,root,parent):
    if root:
        actualTree.create_node(node.value,node.value)
    else:
        actualTree.create_node(node.value,node.value,parent=parent.value)
    if node.isLeaf:
        actualTree.create_node(node.pred,parent=node.value)
    for child in node.children:
        generateTree(child,actualTree,False,node)

        
tree=Tree()
generateTree(root,tree,True,root)
    

In [21]:
tree.show()

Outlook
├── overcast
│   └── ['yes']
├── rain
│   └── Wind
│       ├── strong
│       │   └── ['no']
│       └── weak
│           └── ['yes']
└── sunny
    └── Humidity
        ├── high
        │   └── ['no']
        └── normal
            └── ['yes']

